## First GPT Building Experience

In [53]:
with open('tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [54]:
print(f"Rough estimate of the characters:{len(text)}")

Rough estimate of the characters:1115394


In [55]:
print(f"Text.head():\n{text[:1000]}")

Text.head():
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst f

### Preliminary Test of Characters

In [56]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


### Tokenizing Characters

#### 1. Simple manual tokenizer implementation

In [57]:
# Creating mapping from characters to integers
stoi = { ch: i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


#### 2. Trying tiktoken from OpenAI

In [58]:
import tiktoken
enc = tiktoken.get_encoding('gpt2')
print(enc.encode("hii there"))
print(enc.decode([71, 4178, 612]))

[71, 4178, 612]
hii there


#### For our use with simple characters, let's stick with manual encoding using enumerate which simply assigns each character a value based on their position in the list.
#### Additionally, we use PyTorch to use tensors (multi-dim arrays) for storing the texts in form of datatype = torch.long which signifies that the elements are integers.

In [59]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

## Initial ML Implementation

### 1. Train-Test Split

In [60]:
# 90% Train - 10% Val data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [61]:
block_size = 8 # context length = max size of contiguous characters to sample
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [62]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [63]:
torch.manual_seed(1337)
batch_size = 4 # Number of independent sequences to process in parallel
block_size = 8 # Max context length for prediction

def get_batch(split):
    # Generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [64]:
print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()}, the target is: {target}")

----
when input is [24], the target is: 43
when input is [24, 43], the target is: 58
when input is [24, 43, 58], the target is: 5
when input is [24, 43, 58, 5], the target is: 57
when input is [24, 43, 58, 5, 57], the target is: 1
when input is [24, 43, 58, 5, 57, 1], the target is: 46
when input is [24, 43, 58, 5, 57, 1, 46], the target is: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43], the target is: 39
when input is [44], the target is: 53
when input is [44, 53], the target is: 56
when input is [44, 53, 56], the target is: 1
when input is [44, 53, 56, 1], the target is: 58
when input is [44, 53, 56, 1, 58], the target is: 46
when input is [44, 53, 56, 1, 58, 46], the target is: 39
when input is [44, 53, 56, 1, 58, 46, 39], the target is: 58
when input is [44, 53, 56, 1, 58, 46, 39, 58], the target is: 1
when input is [52], the target is: 58
when input is [52, 58], the target is: 1
when input is [52, 58, 1], the target is: 58
when input is [52, 58, 1, 58], the target is: 46
when in

In [65]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()

        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (Batch ,Time) tensor of integers
        logits = self.token_embedding_table(idx) # (Batch=4, Time=8, Channel=65)

        if targets is None:
            loss = None
        else:
            # implementing negative log likelihood loss

            # first need to reshape for how cross_entropy function wants the input (2-D)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) # expected output = -ln(1/65) = 4.1743872699
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        
        # idx is (Batch, Time) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the preditions
            logits, loss = self(idx)

            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (Batch, Channel)

            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample form the distrubution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

# idx is 1x1 tensor that is holding zero (Zero is newline character)
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [66]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # Typical, 3e-4 is ideal, small 1e-3 is okay. 

In [67]:
batch_size = 32
for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.5727508068084717


In [68]:
print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scace, tridesar, wnl'shenous s ls, theresseys
PlorseelapinghiybHen yof GLUCEN t l-t E:
I hisgothers je are!-e!
QLYotouciullle'z


### Starting with SELF-ATTENTION

In [82]:
# consider the following example

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # Batch, Time, Channels

# for example, element in 5th time stamp will only have access itself and 
# to previous time stamps = 4,3,2,1 and not the future time stamps because
# the idea is to predict the future time stamps. Right now, average of 
# vectors in (4,3,2,1) is considered for the current state. Therefore, it 
# is very lossy. Will fix this later.

x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [83]:
# we want x[b, t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C)) # bow = bag of words (average basically for words)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (Time, Channel) upto Tth token
        xbow[b,t] = torch.mean(xprev, 0) # average over 0th dimension which is t; returns 1-D C vector

#### The above is a very simple implementation with very inefficient code
#### To fix this, we can use matrix multiplication: example provided below.

In [ ]:
a = torch.tril(torch.ones(3, 3))
a = a/torch.sum(a,1,keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

#### We now get averages as we proceed further by simple matrix multiplication hack

In [86]:
# version 2
wei = torch.tril(torch.ones(T,T)) # wei = weights
wei = wei/wei.sum(1, keepdim=True) # a in the example

xbow2 = wei @ x 
# (T,T) @ (B,T,C) -----> Should have been matrix multiplication between these but then will 
# stop because the dimensions are not satisfied for @ and Pytorch will create batch to turn to (B,T,T) @ (B,T,C) which gives 
# ----------> (B,T,C) which is identical to xbow

torch.allclose(xbow, xbow2) # checks if they are same

False

In [90]:
# version 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

# -inf kinda ensures that they don't talk to the future elements

False

In [95]:
# version 4: self attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B,T,16)
q = query(x) # (B,T,16)

wei = q @ k.transpose(-2,-1) # Only want to transpose 16 and T ----> (B,T,16) @ (B,16,T) ----> (B,T,T)

tril = torch.tril(torch.ones(T,T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [93]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089